In [1]:
m = 4
n = 3

3

In [2]:
M = [i for i in 1:m]
F = [j for j in 1:n]

3-element Array{Int64,1}:
 1
 2
 3

In [3]:
using Matching
function mat2vecs{T<:Integer}(prefs::Matrix{T})
    return [prefs[1:findfirst(prefs[:, j], 0)-1, j] for j in 1:size(prefs, 2)]
end

mat2vecs (generic function with 1 method)

In [4]:
m_prefs, f_prefs = mat2vecs.(random_prefs(m, n, allow_unmatched=false))
m_prefs, f_prefs = mat2vecs.(random_prefs(m, n))

(Array{Int64,1}[[1],[2,3,1],[3,2],[2,3]],Array{Int64,1}[[2,1],[4,1,2],[4]])

In [5]:
for o in 1:m
    push!(m_prefs[o],0)
end

for k in 1:n
    push!(f_prefs[k],0)
end

In [6]:
m_prefs

4-element Array{Array{Int64,1},1}:
 [1,0]    
 [2,3,1,0]
 [3,2,0]  
 [2,3,0]  

In [7]:
f_prefs

3-element Array{Array{Int64,1},1}:
 [2,1,0]  
 [4,1,2,0]
 [4,0]    

In [12]:
function matching(m_prefs,f_prefs)

    Z = zeros(m,n) 
    matched_m = zeros(1,m) 
    matched_f = zeros(1,n)
    
    
    for i in 1:m        #男性m人について
     
        if m_prefs[i][1] == 0.0　　#男性iが独身を第一志望するとき
            
            matched_m[i] = 0.0                    
        end
            
        if m_prefs[i][1] !== 0.0  #男性iが第一志望する女性がいるとき

                    
            if sum(Z[:,m_prefs[i][1]]) == 0.0　#男性iが第一志望する女性の列に誰もいないとき
                
                matched_m[i] = m_prefs[i][1]
                matched_f[m_prefs[i][1]] = i
                Z[i,m_prefs[i][1]] = 1.0   
            end
                
            if sum(Z[:,m_prefs[i][1]]) == 1.0 #男性iが第一志望する女性の列に誰かいるとき
                    
                if findfirst(f_prefs[m_prefs[i][1]],i) < findfirst(f_prefs[m_prefs[i][1]],size(Z[:,m_prefs[i][1]],1.0))
                #男性iが受け入れ保留されていた男性よりも女性に好まれているとき
                        
                    matched_m[i] = m_prefs[i][1]
                    matched_f[m_prefs[i][1]] = i
                    Z[i,m_prefs[i][1]] = 1.0
                    Z[size(Z[:,m_prefs[i][1]],1.0),m_prefs[i][1]] = 0.0
                    shift!(m_prefs[size(Z[:,m_prefs[i][1]],1.0)]) #受け入れ保留されていた男性の志望から第一志望を除く
                    
                end                    
                
                else #受け入れ保留されていた男性のほうが女性に好まれているとき
                    
                    shift!(m_prefs[i]) #男性iの志望から第一志望を除く
                    
                end
                
            end
        end
    end

    return matched_m, matched_f

end

LoadError: UndefVarError: matched_m not defined

matching(Any, Any) in module Main at In[11]:3 overwritten at In[12]:3.
